# RateML

### Using RateML to generate a CUDA model file and Pyunicore to excute parameter sweeps on HPC cluster

In [53]:
!git clone https://github.com/DeLaVlag/tvb-root.git
from IPython.display import Markdown, display, Code

In [ ]:
# install tvb-library
%cd tvb-root/scientific_library/
!python setup.py install
# install tvb-bin
%cd ../tvb_bin
!python setup.py install
install tvb-data
!pip install tvb-data
!pip install pylems
%cd ../../

## Building a model
Building rate based models in RateML, start by creating an XML model file. To understand which constructs can be used to build the model, one should take a closer look at the README file. The cell below will prints the latest README file from the repository. Every construct which can be used, is explained.

In [ ]:
mdfile = open("tvb-root/scientific_library/tvb/rateML/README.md","r")
model = mdfile.read()
display(Markdown(model))
mdfile.close()

## Generate a model
After reading the README, one should be able to build an XML model file. Lets use the relatively small Kuramoto model as an example. 
Your model should look like similar to the Kuramoto python file and define some constants, an exposure and dynamics behavior. The dynamics for the Kuramoto consist of a state variable, a derived variable and a time derivative. Except for the derived variable, there are the construct that a RateML XML model file should contain. The template: tvb-root/scientific_library/tvb/rateML/XMLmodels/model_template.xml is an empty template which can be used to create a model XML file.

In [ ]:
# Open the Kuramoto model
model_filename = "kuramoto"
model_location = "tvb-root/scientific_library/tvb/rateML/XMLmodels/"+model_filename+".xml"
!pwd
xmlfile = open(model_location,"r")
model = xmlfile.read()
display(Markdown(model))
xmlfile.close()

## Generating the model code

We will call the templating function in order to automatically generate the model code.
In XML2model.py the class 
```python
RateML('model_filename', language=('python' | 'cuda'), 'path/to/your/XMLmodels', 'path/to/your/generatedModels')
``` 
will start the code generation.

In [43]:
from tvb.rateML.XML2model import RateML

# some preexisting examples:
# model_filename = 'montbrio'
# model_filename = 'oscillator'
model_filename = 'kuramoto'
# model_filename = 'rwongwang'
# model_filename = 'epileptor'
language = "cuda"
XMLin = "tvb-root/scientific_library/tvb/rateML/XMLmodels/"
GenModOut = "tvb-root/scientific_library/tvb/rateML/generatedModels/"
RateML(model_filename, language, XMLin, GenModOut)

2020-11-23 15:44:33,262 - INFO - tvb.rateML.XML2model - True validation of tvb-root/scientific_library/tvb/rateML/XMLmodels/montbrio.xml against https://raw.githubusercontent.com/DeLaVlag/tvb-root/xsdvalidation/scientific_library/tvb/rateML/rML_v0.xsd


In [19]:
from IPython.display import Markdown, display, Code
# Open the generated model
model_location = "tvb-root/scientific_library/tvb/rateML/generatedModels/"+model_filename+".c"
genModFile = open(model_location,"r")
model = genModFile.read()
display(Code(model, language='c'))
# display(Markdown(model))
genModFile.close()

## Simulating the result
If the model displays all its features to your whishes, it is time to take her for a spin on a GPU. The sites that are able to run the models are the JUSUF and JUWELS clusters from Forschungszentrum Juelich. This will only work if you have an LDAP account for these clusters and you are registered in the PyUnicore database. If you dont have access any other CUDA enabled GPU will run your generated model, using \_\_main\_\_.py in rateML/run/ folder .

### Setup PyUnicore

In [20]:
# !pip install pyunicore --upgrade
import pyunicore.client as unicore_client
import json
import os

In [3]:
token = clb_oauth.get_token()
tr = unicore_client.Transport(token)
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)
# r.site_urls
# HPC_LOC = "https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core"
HPC_LOC = 'https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core'
site = unicore_client.Client(transport=tr,site_url=HPC_LOC)

### Transfer model
Transfer the generated model file to JUSUF.

In [44]:
base_url = HPC_LOC + "/storages/PROJECT/"
# base_url = "https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core/storages/PROJECT/"
source_location = "drive/Shared with groups/RateML TVB/tvb-root/scientific_library/tvb/rateML/generatedModels/" + model_filename + ".c"
source_path = os.path.join(os.environ['HOME'], source_location)

storage = unicore_client.Storage(tr, base_url)
storage_location = "wikicollab/RateML/" + model_filename + ".c"
storage.upload(source_path, destination = storage_location)

### Job setup

In [11]:
my_job = {}

# executable / application
# arguments for runthingsJusuf are: backend modelname couplings speeds
my_job['Executable'] = "source /p/project/cslns/wikicollab/RateML/activate; \
cd /p/project/cslns/wikicollab/RateML/; \
./runthingsJuwels "+model_filename+" 32 32"
# ./runthingsJusuf "+model_filename+ " 32 32;"

# environment vars
# run this on login node, not in batch system
my_job['RunOnLoginNode'] = "true"

my_job['Job type'] = "interactive"

# data stage in - TBD
my_job['Imports'] = []

# data stage out - TBD
my_job['Exports'] = []

# Resources - TBD
my_job['Resources'] = {}

### Submit Job to selected HPC cluster 

In [45]:
job = site.new_job(job_description=my_job)

### Info about JOB

In [52]:
if(job.is_running()):
    print('Job is running')

job.poll()
print('Job is finished')

Job is running


### Fetch results
Copy the output log from cluster to Collab

In [49]:
remote = storage.stat("wikicollab/RateML/output.out")
remote.download("output.out")

with open("output.out", "r") as f:
    for line in f:
        print (line.rstrip())

WARNING  File 'cortical' not found in ZIP.
WARNING  File 'hemispheres' not found in ZIP.
WARNING  File 'areas' not found in ZIP.
Finished python simulation successfully in: 68.299
in 0.600 M step/s


Copy the error log from JUSUF to Collab

In [50]:
remote = storage.stat("wikicollab/RateML/error.er")
remote.download("error.er")

with open("error.er", "r") as f:
    for line in f:
        print (line.rstrip())

/p/project/cslns/vandervlag1/cuda_only/tvb-hpc/tvb-root/scientific_library/tvb/datatypes/surfaces.py:61: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)

100%|██████████| 400/400 [01:03<00:00,  6.29it/s]
